# Backpropagation Algorithm

We explore backpropagation assuming the features are $\begin{bmatrix}-10 & 20\end{bmatrix}$ and the correct category is $1$ which is one-hot encoded as $\begin{bmatrix}0 & 1\end{bmatrix}$.

The parameters of the neural network (the weights and biases) which minimize the loss are discovered by descending the loss gradient.

In exercise 530, we calculated the partial derivative of the loss with respect to each parameter using Pytorch's built-in gradient computating function - the '.backward()' method.

In this exercise, we compute those partial derivatives manually, so that we can learn the nuts and bolts of the backpropagation algorithm.

The derivations of the equations we use in this exercise can be found in the Powerpoint slides that accompany this exercise.

In [1]:
import torch
import torch.nn.functional as F

weights = torch.nn.Parameter(torch.Tensor([[1,2],[2,1]]))
print("Parameters (weights): "+str(weights.data))

bias = torch.nn.Parameter(torch.Tensor([[0,0]]))
print("Parameters (bias): "+str(bias.data))

features = torch.autograd.Variable(torch.Tensor([[-10, 20]]))
print("Features: "+str(features.data))

target = torch.autograd.Variable(torch.LongTensor([1]))
#print("The correct class: "+str(target.data))

one_hot_target = torch.autograd.Variable(torch.Tensor([[0, 1]]))
print("One hot encoding of the correct class: "+str(one_hot_target.data))


Parameters (weights): 
 1  2
 2  1
[torch.FloatTensor of size 2x2]

Parameters (bias): 
 0  0
[torch.FloatTensor of size 1x2]

Features: 
-10  20
[torch.FloatTensor of size 1x2]

One hot encoding of the correct class: 
 0  1
[torch.FloatTensor of size 1x2]



We assume the weights are $\begin{bmatrix}1 & 2 \\ 2 & 1\end{bmatrix}$ at start.

The training is assumed to proceed one data point at a time (batch size of 1).

For this iteration of training, the features are $\begin{bmatrix}-10 & 20\end{bmatrix}$ and the correct category is $1$ which is one-hot encoded as $\begin{bmatrix}0 & 1\end{bmatrix}$.

In [2]:
if weights.grad is not None:
    weights.grad.data.zero_()

# Forward pass

result = torch.mm(features, weights) + bias
print("c: "+str(result.data))

softmax_result = F.softmax(result, dim=1)
print("Softmax of c: "+str(softmax_result.data))

log_softmax_result = F.log_softmax(result, dim=1)
print("Log softmax of c: "+str(log_softmax_result.data))

loss_nll_softmax = F.nll_loss(log_softmax_result, target)
print("NLL + log_softmax loss: "+str(loss_nll_softmax.data))

loss = F.cross_entropy(result, target)
print("Cross entropy loss: "+str(loss.data))

# Backward pass

print("transpose of features: "+str(features.data.t()))

grad_c = (softmax_result.data - one_hot_target.data)
print("grad of loss wrt to c: "+str(grad_c))

grad_weights = features.data.t().mm(grad_c)
print("grad of loss wrt to weights: "+str(grad_weights))

grad_bias = grad_c
print("grad of loss wrt to bias: "+str(grad_bias))

print("\tThe manually computed gradient of the loss with respect to weights is "+str(grad_weights))

print("\tThe manually computed gradient of the loss with respect to bias is "+str(grad_bias))

# You can now update the weights and bias

learning_rate = 0.01

weights.data = weights.data - learning_rate * grad_weights

bias.data = bias.data - learning_rate * grad_bias

print("\tThe weights are now "+str(weights.data))

print("\tThe bias is now "+str(bias.data))


c: 
 30   0
[torch.FloatTensor of size 1x2]

Softmax of c: 
 1.0000e+00  9.3576e-14
[torch.FloatTensor of size 1x2]

Log softmax of c: 
-9.2371e-14 -3.0000e+01
[torch.FloatTensor of size 1x2]

NLL + log_softmax loss: 
 30
[torch.FloatTensor of size 1]

Cross entropy loss: 
 30
[torch.FloatTensor of size 1]

transpose of features: 
-10
 20
[torch.FloatTensor of size 2x1]

grad of loss wrt to c: 
 1 -1
[torch.FloatTensor of size 1x2]

grad of loss wrt to weights: 
-10  10
 20 -20
[torch.FloatTensor of size 2x2]

grad of loss wrt to bias: 
 1 -1
[torch.FloatTensor of size 1x2]

	The manually computed gradient of the loss with respect to weights is 
-10  10
 20 -20
[torch.FloatTensor of size 2x2]

	The manually computed gradient of the loss with respect to bias is 
 1 -1
[torch.FloatTensor of size 1x2]

	The weights are now 
 1.1000  1.9000
 1.8000  1.2000
[torch.FloatTensor of size 2x2]

	The bias is now 
1.00000e-03 *
 -10.0000 10.0000
[torch.FloatTensor of size 1x2]



## Parameters

At the end of the first pass, the weights (after being nudged around) should look something like this

$$\begin{bmatrix}1.1 & 1.9 \\ 1.8 & 1.2\end{bmatrix}$$

and the bias should look like this

$$\begin{bmatrix}-0.01 & 0.01\end{bmatrix}$$

You can run the code section above again.

At the end of the second pass, the weights should look like this

$$\begin{bmatrix}1.2 & 1.8 \\ 1.6 & 1.4\end{bmatrix}$$

and the bias should look like this

$$\begin{bmatrix}-0.02 & 0.02\end{bmatrix}$$


## Cross-Check Results

You can check that our computations are correct by verifying the same with Pytorch's automatic backward pass.

In [3]:
# We can uncomment the following two lines to check our gradient against the automatically computed gradients

loss.backward(retain_graph=True)  # Setting retain_graph=True allows us to call loss.backward repeatedly in a local scope

grad_weights = weights.grad.data
print("grad of loss wrt to weights: "+str(grad_weights))

grad_bias = bias.grad.data
print("grad of loss wrt to bias: "+str(grad_bias))

if weights.grad is not None:
    weights.grad.data.zero_()
if bias.grad is not None:
    bias.grad.data.zero_()


grad of loss wrt to weights: 
-10  10
 20 -20
[torch.FloatTensor of size 2x2]

grad of loss wrt to bias: 
 1 -1
[torch.FloatTensor of size 1x2]

